# How Agent Tool Calling Flow Works

## Overview
In this example, we'll explore **how LLMs execute tool calls** at a low level. We'll manually implement the tool-calling flow that libraries like LangChain typically handle automatically.

## Why Learn This?
- **Understanding**: See what happens "under the hood" during tool calling
- **Debugging**: Better troubleshoot issues in production systems

## The Tool Calling Process
When you bind tools to an LLM:
1. **User Query** → LLM receives the question
2. **LLM Reasoning** → Decides which tool(s) to call
3. **Tool Call Request** → LLM returns structured tool call instructions
4. **Tool Execution** → Your code executes the requested tools
5. **Result Return** → Tool outputs are sent back to LLM
6. **Final Response** → LLM generates a natural language answer

In most libraries the **Tool Execution** and **Result Return** are executed in loop automaticcally. Here, we'll implement them manually for educational purposes to understand better the flow.

In [1]:
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
import math
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
import os

# Load environment variables
load_dotenv()

True

## 3. Configure API Key

Set up the OpenAI API key. Make sure you have your API key in a `.env` file or set it directly.

In [2]:
# Set the OpenAI API key from environment variable
# Make sure you have OPENAI_API_KEY in your .env file or uncomment the line below
# os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("⚠️ Warning: OPENAI_API_KEY not found in environment variables")
else:
    print("✓ OpenAI API key loaded successfully")

✓ OpenAI API key loaded successfully


## Tool Definitions

Let's define some tools that can be used by our LLM.  
These are the standard mathematical tools we've used in previous examples.

In [3]:
def compute_factorial(n: int) -> dict:
    """Calculates the factorial of a non-negative integer.
    
    Args:
        n (int): The non-negative integer to compute factorial for
        
    Returns:
        dict: Computation status and result
    """
    print(f"🔧 TOOL CALLED: compute_factorial")
    print(f"   INPUT: n = {n}")
    
    if n < 0:
        result = {"status": "error", "error_message": "Factorial is not defined for negative numbers."}
        print(f"   OUTPUT: {result}")
        return result
    
    if n > 170:
        result = {"status": "error", "error_message": "Number too large. Factorial computation limited to n <= 170."}
        print(f"   OUTPUT: {result}")
        return result
    
    try:
        factorial_result = 1
        for i in range(2, n + 1):
            factorial_result *= i
        
        result = {"status": "success", "result": f"The factorial of {n} is {factorial_result}"}
        print(f"   OUTPUT: {result}")
        return result
    except Exception as e:
        result = {"status": "error", "error_message": f"Error computing factorial: {str(e)}"}
        print(f"   OUTPUT: {result}")
        return result

def compute_exponential(base: float, exponent: float) -> dict:
    """Calculates base raised to the power of exponent.
    
    Args:
        base (float): The base number
        exponent (float): The exponent
        
    Returns:
        dict: Computation status and result
    """
    print(f"🔧 TOOL CALLED: compute_exponential")
    print(f"   INPUT: base = {base}, exponent = {exponent}")
    
    try:
        result_value = base ** exponent
        
        if abs(result_value) > 1e308:
            result = {"status": "error", "error_message": "Result too large to compute."}
            print(f"   OUTPUT: {result}")
            return result
        
        result = {"status": "success", "result": f"{base} raised to the power of {exponent} is {result_value}"}
        print(f"   OUTPUT: {result}")
        return result
    except Exception as e:
        result = {"status": "error", "error_message": f"Error computing exponential: {str(e)}"}
        print(f"   OUTPUT: {result}")
        return result

def compute_fibonacci(n: int) -> dict:
    """Calculates the nth Fibonacci number (0-indexed).
    
    Args:
        n (int): The position in the Fibonacci sequence (0-indexed)
        
    Returns:
        dict: Computation status and result
    """
    print(f"🔧 TOOL CALLED: compute_fibonacci")
    print(f"   INPUT: n = {n}")
    
    if n < 0:
        result = {"status": "error", "error_message": "Fibonacci is not defined for negative positions."}
        print(f"   OUTPUT: {result}")
        return result
    
    if n > 1000:
        result = {"status": "error", "error_message": "Position too large. Fibonacci computation limited to n <= 1000."}
        print(f"   OUTPUT: {result}")
        return result
    
    try:
        if n == 0:
            fib_result = 0
        elif n == 1:
            fib_result = 1
        else:
            a, b = 0, 1
            for _ in range(2, n + 1):
                a, b = b, a + b
            fib_result = b
        
        result = {"status": "success", "result": f"The {n}th Fibonacci number is {fib_result}"}
        print(f"   OUTPUT: {result}")
        return result
    except Exception as e:
        result = {"status": "error", "error_message": f"Error computing Fibonacci: {str(e)}"}
        print(f"   OUTPUT: {result}")
        return result

print("✓ Mathematical tools defined successfully")

✓ Mathematical tools defined successfully


## System Prompt and Tool Binding

We define now the system prompt that will be used by the LLM.
We also define a **Tool Dictionary**: Maps tool names to actual functions that will be helpful for manual execution.

In [4]:
math_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a numeric computation specialist. 
You can calculate factorial, exponential (power), and Fibonacci numbers. 
When the user asks for computations, use the appropriate tool:
- For factorial, use `compute_factorial`
- For exponential (power), use `compute_exponential`
- For Fibonacci, use `compute_fibonacci`
"""),
("user", "{query}")
])

available_tools = {
    "compute_factorial": compute_factorial,
    "compute_exponential": compute_exponential,
    "compute_fibonacci": compute_fibonacci
}

math_llm = ChatOpenAI(model="gpt-5-mini").bind_tools(available_tools.values())

## LLM Invocation (First Call)

Now we'll invoke the LLM with a query that requires tool usage.  


The **Expected Behavior** if the following:
- `response.content` will be empty (or minimal)
- `response.tool_calls` will contain the tool call request(s)

In [5]:
query = "Tell me the value of fibonacci of 8"

messages = math_prompt.invoke({"query": query})
response = math_llm.invoke(messages)

print(response)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 291, 'total_tokens': 379, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CjS2SBjkXoIiGysBG4LM6dY2hbshU', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--fce03c3b-aeb6-40c3-ac9a-0494ffc77d63-0' tool_calls=[{'name': 'compute_fibonacci', 'args': {'n': 8}, 'id': 'call_eqx44drrcbtZSihUOYxq7HDy', 'type': 'tool_call'}] usage_metadata={'input_tokens': 291, 'output_tokens': 88, 'total_tokens': 379, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}}


### Response Analysis
In the response you should have something like this where the content is empty but the llm ask you to call a tool

<pre><span>AIMessage</span>(
  <span>content</span>=<span class="string-value">''</span>,
  ...............
  <div class="highlight-tool-calls">
  <b>tool_calls:</b>
  [
    {
      '<span class="key">name</span>': <span class="string-value">'compute_fibonacci'</span>,
      '<span class="key">args</span>': {'<span class="key">n</span>': <span class="number-value">8</span>},
      '<span class="key">id</span>': <span class="string-value">'call_XXXXXXXXXXXXXXXXX'</span>
    }
  ]
  </div>
  ..............
)</pre>


**Key Observations:**
- **content = ''**: LLM hasn't provided a final answer yet so the content is empty
- **tool_calls**: List of tools the LLM wants to execute


## Execute Tool Calls Manually

Now we manually execute the tools that the LLM requested.

**The Manual Process:**
1. Loop through each tool call in `response.tool_calls`
2. Extract the tool name and arguments
3. Look up the actual function from our `available_tools` dictionary
4. Execute the function with the provided arguments
5. Collect the results

**Why Manual Execution?**
- In production libraries, this is automatic
- Doing it manually helps you understand the flow
- Allows custom error handling and logging
- Enables tool execution optimization (parallel execution, caching, etc.)

In [6]:
tool_results = []

for tool_call in response.tool_calls:
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]
    
    print(f"   EXECUTING: {tool_name} with args {tool_args}")
    
    # Call the actual function
    tool_function = available_tools[tool_name]
    tool_output = tool_function(**tool_args)
    
    tool_results.append(tool_output) 

   EXECUTING: compute_fibonacci with args {'n': 8}
🔧 TOOL CALLED: compute_fibonacci
   INPUT: n = 8
   OUTPUT: {'status': 'success', 'result': 'The 8th Fibonacci number is 21'}


### Ouput analysis

You should have an output like the following:

<pre>
 <b>OUTPUT:</b> {'status': 'success', 'result': 'The 8th Fibonacci number is 21'}
</pre>
We have manually performed what libraries typically handle automatically: tool invocation and iteration. In this example, iteration is not applied, but it’s important to note that you could iterate to generate the final output.  
For demonstration purposes, in the example, we will perform just two iterations. Instead of calling tools multiple times, we make a single tool call, pass the result back to the LLM, and continue the process.

The iteration is a loop that executes until a stop condition is reached. A stop condition could be:
- When `response.content` is not empty and `response.tool_calls` is empty, or
- When a maximum number of iterations has been reached.
Including a maximum iteration limit is essential, since an LLM may otherwise fall into infinite loops. Most libraries enforce such a limit by default to ensure safe and predictable execution.


## Second Iteration: Generate Final Response

After generating the initial response, we should iterate once more to obtain the final response.  
In this case, the iteration should produces a non‑empty `response.content` so the stop condition is reached and the result can be returned to the client.

In [7]:
messages_with_results = [
    messages.messages[0],  # system message
    messages.messages[1],  # user query
    response,  # LLM's tool call request
    {"role": "tool", "content": str(tool_results), "tool_call_id": response.tool_calls[0]["id"]}
]

# Get final answer from LLM
final_response = math_llm.invoke(messages_with_results)

In [8]:
if final_response.content:
    print(f"Final response is: {final_response.content}")
else:
    print("Strange result at this point the model should returns the answer, without asking other iterations")

Final response is: The 8th Fibonacci number (0-indexed) is 21.
